# pré-processamento

In [58]:
import pandas as pd
import tabula

In [59]:
# df = pd.read_csv("C:/Users/sonia/Downloads/Extrato conta - 012024.csv", encoding='ISO-8859-1')
df = pd.read_csv("C:/Users/sonia/Downloads/Extrato conta - 022024.csv", encoding='ISO-8859-1')


In [60]:
dfs = tabula.read_pdf('c:/Users/sonia/Downloads/DOC-20240211-WA0002_240211_165932.pdf',pages=2)

In [61]:
dfs = dfs[0]

In [62]:
# Supondo que dfs é o seu DataFrame
# Substitua 'dfs' pelo nome real do seu DataFrame

# Selecione as colunas que você quer unir
colunas_1 = ['Unnamed: 0', '1TOMAZ S SOUZA CRUZCartão N. 6194', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
colunas_2 = ['10/12', 'SUPERMERCADOS ANGELONI FLORIANOPOLIS', 'BR', 'R$', '57,04']

# Selecione as linhas que você quer unir
linhas_1 = dfs[colunas_1]
linhas_2 = dfs[colunas_2]

# Adicione um novo índice a partir da segunda linha
linhas_2.index += 1

# Renomeie as colunas para que tenham o mesmo nome
linhas_2.columns = colunas_1

# Use a função concat para unir as linhas
resultado = pd.concat([linhas_1, linhas_2])

# Exemplo de impressão do novo DataFrame
#resultado

In [63]:
resultado.rename(columns={'Unnamed: 0':'Data', '1TOMAZ S SOUZA CRUZCartão N. 6194':'Descrição', 'Unnamed: 2':'ID', 'Unnamed: 3':'Moeda', 'Unnamed: 4':'Valor'}, inplace=True)


In [64]:
resultado['Valor'] = resultado['Valor'].str.replace('.', '').str.replace(',', '.').astype(float)


In [65]:
df['Valor'] = df['Valor'].str.replace('.', '').str.replace(',', '.').astype(float)

# Dataframes

In [66]:
receitas = df[df['Tipo Lançamento']=='Entrada']
receitas = receitas[['Data','Lançamento','Detalhes','Valor']]

In [67]:
gastos = df[df['Tipo Lançamento']=='Saída']
gastos = gastos[['Data','Lançamento','Detalhes','Valor']]

In [68]:
gastos_creditos = resultado[(resultado['Valor']>0)&(resultado['Data'].notnull())]

In [69]:
# Crie uma máscara booleana usando str.contains
mascara_pagamento = gastos['Lançamento'].str.contains('pagamento', case=False, na=False)
mascara_pgto = gastos['Lançamento'].str.contains('pagto', case=False, na=False)
mascara_faxina = gastos['Detalhes'].str.contains('Rita',case=False,na=False)
mascara_animais = gastos['Detalhes'].str.contains('Animais',case=False,na=False)


# Aplique a máscara ao DataFrame original
gastos_fixos = gastos[mascara_pagamento|mascara_pgto|mascara_faxina|mascara_animais]

In [70]:
qualidade_vida = gastos[~mascara_pagamento&~mascara_pgto&~mascara_faxina&~mascara_animais]

In [71]:
receita_total = receitas['Valor'].sum()
gastos_qualidade_vida = -qualidade_vida['Valor'].sum()
gastos_gastos_fixos = -gastos_fixos['Valor'].sum()
gastos_investimentos = receita_total - gastos_qualidade_vida - gastos_gastos_fixos
meta_qualidade_vida = 0.4
meta_gastos_fixos = 0.4
meta_investimento = 0.2

In [72]:
# Criando o DataFrame 'relatorio'
relatorio = pd.DataFrame({
    'Descrição': ['Investimentos', 'Qualidade de Vida', 'Gastos Fixos'],
    'Acumulado': [gastos_investimentos, gastos_qualidade_vida, gastos_gastos_fixos],
    'Meta': [meta_investimento * receita_total, meta_qualidade_vida * receita_total, meta_gastos_fixos * receita_total]
})

In [73]:
relatorio['Diferença'] = relatorio['Meta'] - relatorio['Acumulado']
relatorio['%Diferença'] = round(100*relatorio['Diferença']/relatorio['Meta'],2)
relatorio

,Descrição,Acumulado,Meta,Diferença,%Diferença
0,Investimentos,2204.73,1080.458,-1124.272,-104.06
1,Qualidade de Vida,1000.79,2160.916,1160.126,53.69
2,Gastos Fixos,2196.77,2160.916,-35.854,-1.66
